In [ ]:
#Importing libraries for data management
import pandas as pd
import numpy as np

#Importing libraries for system management
import os

#Distance computation
from pyjarowinkler import distance
import Levenshtein

In [ ]:
#TODO: Check if Danish computers have the same problem...
I_want_to_run_all_the_preprocessing = False

# Converting the current format (ISO-8859-14) into UTF-8 
iconv -f ISO-8859-14 file_in.csv > file_out.tsv

In [ ]:

if I_want_to_run_all_the_preprocessing:
    
    #Copying into a new directory
    ! cp -R ../../city_dump/data/LL/ ../../city_dump/data/utf8/

    #Getting all files
    d_path = '../../city_dump/data/utf8/'
    l = next(os.walk(d_path))[2]

    #Converting the files to utf8 (not sure this will work in computers different than mac...)
    for file in l:
        f_path = d_path+file
        tmp_path = d_path+'tmp.csv'
        ! iconv -f ISO-8859-14 $f_path > $tmp_path
        !mv $tmp_path $f_path

# Reading DigDag
and transforming it so I encode as many logical variation of the names as possible. In this way the variations for each place is only appied once whereas if we do the same for the data it will need to be applied to each place taking much longer. But the way to remember the original variation is to keep the "enhedid", which is the reference id.

In [ ]:
# This function adds extra rows in the dataframe *dd* for the 
# places where *word* exists and replaces it with *replacement*
def adding_extra_rows(dd, word, replacement):
    dd['special'] = dd['simplename'].apply(lambda x: word in x) 
    dd['simplename2'] = dd.simplename.apply(lambda x: x.replace(word,replacement))
    return pd.concat([dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename']],dd[dd['special']][['navn', 'enhedid', 'enhedtype', 'art', 'simplename2']].rename(columns={'simplename2':'simplename'})])

In [ ]:
#The digdag seems to have duplicates which I do not understand why
dd = pd.read_csv('../../city_dump/data/rl_places_digdag_v1.txt', sep = '\t', encoding='utf-16', dtype=str)
dd_org = dd.copy()

#Getting the Købstad and putting it to the original data
dd2 = pd.read_csv('../../city_dump/data/koebstad.csv', sep = ';', encoding='utf-8', dtype=str)
dd2['enhedtype'] = dd2['art'] 
dd = pd.concat([dd, dd2[~dd2.isna()][['navn','enhedid','enhedtype','art','simplename']]])
dd['simplename'] = dd['simplename'].astype(str)


#Moving the Købstad to the right column
dd['art'] = np.where(dd['enhedtype'] == 'Købstad', ['Købstadskommune']*len(dd), dd['art'])

#adding a conversion of special characters to latin letters (å -> aa, ø -> oe, æ -> ae) and adding them to the reference list
dd = adding_extra_rows(dd, 'å', 'aa')
dd = adding_extra_rows(dd, 'ø', 'oe')
dd = adding_extra_rows(dd, 'æ', 'ae')

#Preparing the names to append them at the simple name
dd['art'] = dd['art'].apply({'Amt':'amt', 'Sogn':'sogn','Købstadskommune':'købstad', 'Geografisk Herred':'herred', 'Processing':''}.get)

#Adding a duplicated list of their unit type to increase matches
dd['simplename2'] = dd.apply(lambda row: str(row['simplename'])+' '+str(row['art']), axis=1)
dd = pd.concat([dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename']],dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename2']].rename(columns={'simplename2':'simplename'})])

#Adding the plural købstæder as well
dd = adding_extra_rows(dd, 'købstad', 'købstæder')

#Adding spaces instead of hyphens (new rows)
dd = adding_extra_rows(dd, '-', ' ')

#Removing spaces (new rows)
dd = adding_extra_rows(dd, ' ', '')

#Adding bysogn and landsogn (new rows)
dd = adding_extra_rows(dd, 'sogn', 'bysogn')
dd = adding_extra_rows(dd, 'sogn', 'landsogn')

#adding rows where the last letter's "e" will be removed
dd['special'] = dd['simplename'].str[-1] == 'e'
dd['simplename2'] = dd['simplename'].str[:-1]
dd = pd.concat([dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename']],dd[dd['special']][['navn', 'enhedid', 'enhedtype', 'art', 'simplename2']].rename(columns={'simplename2':'simplename'})])

#Dropping duplicates from digdag
#print(dd[dd.duplicated('simplename', keep=False)].sort_values('simplename').head(30))
dd.drop_duplicates(['art','simplename'], keep='first', inplace=True)

#Drop 'Non' from the list, this may be one of my artifacts...
dd = dd[(dd.simplename.apply(lambda x: not 'Non' in x))]

# TODO take into consideration that the same city name can be in varios geographical areas...
# It should be matched with herred to increase accuracy for each record. However, right now we're only cleaning
# not sure how pressing the issue is...
nr_sogn = len(dd_org[dd_org.art=='Sogn'].simplename.unique()) 
dd.head(20)

#TODO: Right now I can map things to amt for example which should not be the case...
#dd =dd[dd.art='sogn']

In [ ]:
print('length original:\t', len(dd_org), '\nlength modified:\t', len(dd), '\nunique sogn keys:\t', nr_sogn)

# Processing functions

In [ ]:
#Get the df from the $ separated file (for some reason pandas has problems with random lines)
def get_df(f_path):
    r= []
    columns = []
    with open(f_path) as f:
        first = True
        for line in f:
            line = line.rstrip().split('$')
            if first:
                length = len(line)
                columns = line
                first=False
            else:
                r.append(line[:length])
                
    return pd.DataFrame(data=r, columns = columns)

In [ ]:
# Name cleaning function
def name_cleaner(s):
    try:
        o =s[working_column].lower().rstrip().replace('  ', ' ')#.replace(' købstad', '')
        if ' (' in o: return o.split(' (')[0]
        return o
    except:
        return np.nan

# Grouping all 1901 files together

In [ ]:
if I_want_to_run_all_the_preprocessing:
    
    l =['ft1901_LL_aalborg.txt',
     'ft1901_LL_aarhus.txt',
     'ft1901_LL_bornholm.txt',
     'ft1901_LL_frederiksborg.txt',
     'ft1901_LL_hjoerring.txt',
     'ft1901_LL_holbaek.txt',
     'ft1901_LL_kbhv.txt',
     'ft1901_LL_maribo.txt',
     'ft1901_LL_odense.txt',
     'ft1901_LL_praestoe.txt',
     'ft1901_LL_randers.txt',
     'ft1901_LL_ribe.txt',
     'ft1901_LL_ringkoebing.txt',
     'ft1901_LL_roskilde.txt',
     'ft1901_LL_skanderborg.txt',
     'ft1901_LL_soroe.txt',
     'ft1901_LL_svendborg.txt',
     'ft1901_LL_thisted.txt',
     'ft1901_LL_vejle.txt',
     'ft1901_LL_viborg.txt']

    _path = '../../city_dump/data/utf8/'

    df_list = []
    for f in l:

        print(f)
        #Loading the data
        df_list.append(get_df(_path+f))

    #Concatenating all the files
    df_list = pd.concat(df_list, sort=False)

    #Saving the unique file
    df_list.to_csv(_path+'ft1901_LL.txt', sep='$', index=False)

In [ ]:
# List of files to work (with the 1901 collapsed)
working_data = ['ft1845_LL.txt',
 'ft1850_LL.txt',
 'ft1860_LL.txt',
 'ft1880_LL.txt',
 'ft1885_LL.txt',
 'ft1901_LL.txt']

# Basic matching
(only performs the match if the names are identical)

_This could be done only once by countring reference id and year. Then with the names we can keep constraining with matches_

In [ ]:
_path = '../../city_dump/data/utf8/'

#This is where I save the counts and then group it together
out_list = []

#where I save the missmatches and file year
missmatches = []
total_records = 0

for f in working_data:
    
    print(f)
    #Loading the data
    df = get_df(_path+f)
    
    #Dropping empty rows due to the "mal-conversion" to utf8... The empty rows do contain data originally
    #TODO: Fix the conversion thing (ask Barbara to provide me the UTF-8...)
    df = df[~df.Herred.isna()]
    print(len(df))
    total_records = total_records + len(df)
    
    #Preforming the name cleaning for Sogne, Herred, and Amt
    for working_column in ['Sogne','Herred','Amt']:
        df[working_column] = df.apply(lambda x: name_cleaner(x), axis=1)

    # Focusing on Sogn that match the Sogn list in DigDag
    out = df.merge(dd[dd.art=='sogn'][['simplename', 'art', 'enhedid']], left_on='Sogne', right_on='simplename', how='left')

    #keeping the missmatch
    miss = out[out.art.isna()].drop_duplicates('Sogne', keep='first')
    miss['year'] = f[2:6]
    missmatches.append(miss)
    
    #Counting the matches
    out = out[~out.art.isna()].groupby('enhedid').size().reset_index(name='counts')
    
    # setting the year
    out['year'] = f[2:6]
    
    #Keeping the counts on RAM
    out_list.append(out)
    

#Concatenating all the files (the counts and missmatches)
out_list = pd.concat(out_list, sort=False)
missmatches = pd.concat(missmatches, sort=False)

#Saving the unique file in the HD
out_list = out_list.merge(dd_org, on ='enhedid').pivot('enhedid', 'year', 'counts').fillna(0).astype(int)
out_list.to_csv(_path+'../out/counts.txt', sep='$')
out_list.to_csv(_path+'../out/places_FT_uniquevalues_01.tsv', sep='\t')

print('Done :D')

nr_matched_records = out_list.sum().sum() #Two times sum, first one for year, and second one among years
print('Matched records:',nr_matched_records, 'out of', total_records, '(', nr_matched_records/total_records*100,')%')
print('Matched places:', len(out_list), 'out of', nr_sogn, '(', len(out_list)/nr_sogn*100,')%')


In [ ]:
#Cities that could not be matched on the first round counts per year
missmatches.groupby('year').size().reset_index(name='counts').sort_values('counts', ascending=False)

# Computing city similarity for all missmatches

In [ ]:
#Getting the potential first matches

mm = missmatches.Sogne.unique()
sn = dd.simplename.unique() #Another possibility wowuld be to match it to the original (shorter)

distance_jaro = np.zeros((len(sn),len(mm)))
distance_leven = np.zeros((len(sn),len(mm)))

for i in range(len(sn)):
    if i%100 == 0: print(i, 'out of' , len(sn))
        
    for j in range(len(mm)): #The internal loop could be done using apply (it should be the longer one)
        #If variable is none skip
        if not mm[j]: continue
        
        #This matrix is not simetric because x and y axis are not the same!
        try:
            distance_jaro[i][j] = distance.get_jaro_distance(sn[i],mm[j])
            distance_leven[i][j] = Levenshtein.distance(sn[i],mm[j])/(len(sn[i])+len(mm[j]))
        except:
            print('Could not make it',i,j)
            print(sn[i],mm[j])
        
#Index is the reference names found in the data, columns to the digdag
distance_jaro = pd.DataFrame(data = distance_jaro, columns = mm, index = sn)
distance_leven = pd.DataFrame(data = distance_leven, columns = mm, index = sn)
print(distance_jaro.shape) #(26428, 1237)
distance_jaro.head()

In [ ]:
dd_org[dd_org.simplename == 'stilling'] #Stilling in dd_org!!

In [ ]:
'stilling' in distance_jaro.index

In [ ]:
#Saving the best possible match (left=name found in the data, right=name found in the extended version of digdag)
a = distance_jaro.idxmax().reset_index(name = 'potential_match')
b = distance_jaro.max().reset_index(name='jaro')
c = a.merge(b, on='index').rename(columns={'index':'data_name'})
c.to_csv(_path+'../out/mapping.tsv', sep='\t')
c.head()

# Advanced matching
(This is the same as the basic matching but in this case it matches to the computed best match)

In [ ]:
#Select threshold to perform the matching, each mapping contains only the best jaro
THRESHOLD = 0.9

dfmap = pd.read_csv(_path+'../out/mapping.tsv', sep = '\t', dtype=str, names=['original','mapped', 'score'], skiprows=1)
dfmap = dfmap[dfmap.score.astype(float) >= THRESHOLD][['original','mapped']]
dfmap.head()

In [ ]:
_path = '../../city_dump/data/utf8/'

#This is where I save the counts and then group it together
out_list = []


for f in working_data:
    
    print(f)
    #Loading the data
    df = get_df(_path+f)
    
    #Dropping empty rows due to the "mal-conversion" to utf8... The empty rows do contain data originally
    #TODO: Fix the conversion thing
    df = df[~df.Herred.isna()]
    print(len(df))
    
    #Preforming the name cleaning for Sogne, Herred, and Amt
    for working_column in ['Sogne','Herred','Amt']:
        df[working_column] = df.apply(lambda x: name_cleaner(x), axis=1)
    
    #Performing the matching for "inexisting places"
    df = df.merge(dfmap, left_on='Sogne', right_on='original', how='left')
    df.loc[~df.mapped.isna(), 'Sogne'] = df.loc[~df.mapped.isna(), 'mapped']


    # Focusing on Sogn that match the Sogn list in DigDag
    out = df.merge(dd[dd.art=='sogn'][['simplename', 'art','enhedid']], left_on='Sogne', right_on='simplename', how='left')
    
    #Counting the matches
    out = out[~out.art.isna()].groupby('enhedid').size().reset_index(name='counts')
    
    # setting the year
    out['year'] = f[2:6]
    
    #Keeping the counts on RAM
    out_list.append(out)

    
#Concatenating all the files (the counts and missmatches)
out_list = pd.concat(out_list, sort=False)


#Saving the unique file in the HD
out_list = out_list.merge(dd_org, on ='enhedid').pivot('simplename', 'year', 'counts').fillna(0).astype(int)
out_list.to_csv(_path+'../out/counts_first_jaro.txt', sep='$')
out_list.to_csv(_path+'../out/places_FT_jaro0.9_01.tsv', sep='\t')


print('Done :D')

nr_matched_records = out_list.sum().sum() #Two times sum, first one for year, and second one among years
print('Matched records:',nr_matched_records, 'out of', total_records, '(', nr_matched_records/total_records*100,')%')
print('Matched places:', len(out_list), 'out of', nr_sogn, '(', len(out_list)/nr_sogn*100,')%')


In [ ]:
#Adding few more requested variables
dfmap = pd.read_csv(_path+'../out/mapping.tsv', sep = '\t', dtype=str, names=['original','mapped', 'score'], skiprows=1)
jaro09 = pd.read_csv(_path+'../out/places_FT_jaro0.9_01.tsv', sep='\t')
jaro09 = jaro09.merge(dfmap[dfmap.score.astype(float) >= THRESHOLD], left_on='simplename', right_on='mapped').merge(dd[['simplename', 'art', 'enhedid']], on='simplename')
jaro09.to_csv(_path+'../out/places_FT_jaro0.9_01.tsv', sep='\t')
jaro09.head()

In [ ]:
# Finding the best possible matches

concatenate = []
i=0
for col in distance_jaro.columns:
    i = i+1
    if i % 100 == 0: print(i, 'out of', len(distance_jaro.columns))
    aux = distance_jaro.nlargest(5, col)[[col]]
    aux.columns = ['score']
    aux['original'] = col
    concatenate.append(aux)

concatenate = pd.concat(concatenate, sort=False)
concatenate = concatenate.reset_index()
concatenate.columns = ['potential_match','score','original']
concatenate[['original','potential_match','score']][~concatenate.original.isin(concatenate[concatenate.score >= 0.9].original.unique())].to_csv(_path+'../out/places_possible_matches_jaro_01.tsv', index=False, sep='\t')

concatenate = pd.read_csv(_path+'../out/places_possible_matches_jaro_01.tsv',  sep='\t')
concatenate.head(15)

# Getting the counts for all possible matches (reading only once)

In [ ]:
_path = '../../city_dump/data/utf8/'

#This is where I save the counts and then group it together
out_list = []


for f in working_data:
    
    print(f)
    #Loading the data
    df = get_df(_path+f)
    
    #Dropping empty rows due to the "mal-conversion" to utf8... The empty rows do contain data originally
    #TODO: Fix the conversion thing
    df = df[~df.Herred.isna()]
    print(len(df))
    
    #Preforming the name cleaning for Sogne, Herred, and Amt
    for working_column in ['Sogne','Herred','Amt']:
        df[working_column] = df.apply(lambda x: name_cleaner(x), axis=1)
    
    df['year'] = f[2:6]
    
    #TODO: Do the counting inside so it removes overhead
    
    #Keeping the counts on RAM
    out_list.append(df)

    
#Concatenating all the files (the counts and missmatches)
df = pd.concat(out_list, sort=False)
df.head()

In [ ]:
#Read everything and now do the counting outside (not the best... would be better to count inside the loop)
dfc = df.groupby(['Sogne','year']).size().reset_index(name='counts')
dfc.head()

In [ ]:
THRESHOLD = 0.9

#One row per city
a = dfc.rename(columns={'year':'id'}).pivot(index='Sogne', columns='id', values='counts').reset_index().fillna(0).astype(int, errors='ignore')

#Performing the matching for mapped places
dfmap = pd.read_csv(_path+'../out/mapping.tsv', sep = '\t', dtype=str, names=['original','mapped', 'score'], skiprows=1)
dfmap =  dfmap[dfmap.score.astype(float) >= THRESHOLD]

df = a.merge(dfmap, left_on='Sogne', right_on='original', how='left')
df.loc[~df.mapped.isna(), 'Sogne'] = df.loc[~df.mapped.isna(), 'mapped']

# Focusing on Sogn that match the Sogn list in DigDag
df = df.merge(dd[dd.art=='sogn'][['simplename', 'art','enhedid']], left_on='Sogne', right_on='simplename', how='left')
df.loc[df.simplename.isna() & ~df.mapped.isna(), 'simplename'] = df.loc[df.simplename.isna() & ~df.mapped.isna(), 'mapped']
df.loc[~df.simplename.isna(), 'Sogne'] = df.loc[~df.simplename.isna(), 'simplename']

df.head(20)

In [ ]:
#Summing the mapped columns
cols = df.columns
df = df.merge(df.groupby(['Sogne'],as_index = False).agg({'1845':'sum', '1850':'sum','1860':'sum', '1880':'sum','1885':'sum', '1901':'sum'}), on='simplename')
df.loc[~df['1845_y'].isna(), '1845'] = df.loc[~df['1845_y'].isna(), '1845_y']
df.loc[~df['1845_y'].isna(), '1850'] = df.loc[~df['1845_y'].isna(), '1850_y']
df.loc[~df['1845_y'].isna(), '1860'] = df.loc[~df['1845_y'].isna(), '1860_y']
df.loc[~df['1845_y'].isna(), '1880'] = df.loc[~df['1845_y'].isna(), '1880_y']
df.loc[~df['1845_y'].isna(), '1885'] = df.loc[~df['1845_y'].isna(), '1885_y']
df.loc[~df['1845_y'].isna(), '1901'] = df.loc[~df['1845_y'].isna(), '1901_y']
df = df[cols]

In [ ]:
cols

In [ ]:
z = concatenate.merge(df, left_on='original', right_on='Sogne').merge(df, left_on='potential_match', right_on='Sogne')
for y in ['1845', '1850', '1860', '1880', '1885', '1901']:
    z[y] = z[y+'_x'] + z[y+'_y']
z = z[['original','potential_match', 'score', '1845', '1850', '1860', '1880', '1885', '1901', 'art', 'enhedid']].merge(dd.drop_duplicates('simplename', keep='first'), left_on='potential_match', right_on='simplename')

z.head()

In [ ]:
# z should have what I want